In [1]:
import os

os.environ['KERAS_BACKEND'] = "torch"

In [2]:
import embeddings
import keras


target_file = "./../data/embeddings-target.o"
corpus_file = "./../data/corpus.preprocessed.csv"

# Gensim Word2Vec

In [ ]:

# TODO: Correctly parametrize: STUDY THE BEST PARAMETERS?
emb_model = embeddings.WordEmbedding(7200, 128, target_file=target_file, corpus_file=corpus_file)
emb_model.load_model()

In [ ]:
emb_model.model.wv.most_similar("agency")

In [ ]:
print(f"We have a total of words: {len(emb_model.model.wv.key_to_index)}")

In [ ]:
# TODO: Should I also filter out the names of the games? Do I loose on value if I keep them?

# My Word2Vec Implementation

In [ ]:
my_word2vec = embeddings.MyWord2Vec(25000, 128, "")

In [ ]:
import dataset
from torch.utils.data import DataLoader

ds = dataset.CommentDataset(corpus_file)

training_dataloader = DataLoader(ds, batch_size=32, shuffle=True, collate_fn=lambda x: x)
# TODO: Convert to numbers (indices of a vocabulary) as model doesnt do that by itself
my_word2vec.compile(optimizer="SGD", loss="categorical_crossentropy", metrics=["acc"])
my_word2vec.fit(training_dataloader, epochs=10)

Pandas Apply:   0%|          | 0/1939904 [00:00<?, ?it/s]

In [7]:
ds.dataset.itertuples()

NameError: name 'ds' is not defined